In [26]:

import os
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras import backend as K
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.utils import to_categorical

In [24]:
def load_data(data_dir, img_size=(224, 224)):
    X = []
    y = []
    class_names = os.listdir(data_dir)
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(class_index)
    return np.array(X), np.array(y), class_names

In [13]:

def create_model(num_classes, embedding_dim=128):
    base_model = ResNet50(weights='imagenet', include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Embedding layer
    embedding = Dense(embedding_dim, activation=None, name='embedding')(x)
    embedding_normalized = Lambda(lambda x: K.l2_normalize(x, axis=1), name='embedding_normalized')(embedding)
    
    # Fully connected layers for classification
    fc = Dense(1024, activation='relu')(embedding_normalized)
    fc = Dense(512, activation='relu')(fc)
    output = Dense(num_classes, activation='softmax', name='classification')(fc)
    
    model = Model(inputs=base_model.input, outputs=[output, embedding_normalized])
    
    return model

In [14]:

def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[:, 0:128], y_pred[:, 128:256], y_pred[:, 256:]
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    neg_dist = K.sum(K.square(anchor - negative), axis=1)
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum
    (basic_loss, 0.0)
    return K.mean(loss)


In [15]:
def train_model(model, train_dir, val_dir, epochs=10, batch_size=32):
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    model.compile(
        optimizer='adam',
        loss={
            'classification': 'categorical_crossentropy',
            'embedding_normalized': triplet_loss
        },
        loss_weights={
            'classification': 1.0,
            'embedding_normalized': 0.5
        },
        metrics={
            'classification': ['accuracy', TopKCategoricalAccuracy(k=1, name='top_1_accuracy')]
        }
    )
    
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        epochs=epochs
    )
    
    return model, history



In [16]:
def calculate_embeddings(model, X):
    return model.predict(X)[1]  # Return the embedding output

def calculate_similarity(embeddings1, embeddings2):
    return cosine_similarity(embeddings1, embeddings2)




In [18]:
# Step 4: Face detection and recognition
def detect_and_recognize(image_path, model, class_names):
    face_cascade = cv2.CascadeClassifier(r"C:\Users\hp\Desktop\ml\haarcascade_frontalface_default.xml")
    img = cv2.imread(r"C:\Users\hp\Desktop\ml\what.jpeg")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        face = np.expand_dims(face, axis=0)
        face = preprocess_input(face)
        
        prediction = model.predict(face)
        class_index = np.argmax(prediction)
        class_name = class_names[class_index]
        confidence = prediction[0][class_index]
        
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(img, f"{class_name} ({confidence:.2f})", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    cv2.imshow('Face Recognition', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [27]:
def evaluate_model(model, X_test, y_test, num_classes):
    y_test_cat = to_categorical(y_test, num_classes)
    y_pred, embeddings = model.predict(X_test)
    
    # Calculate top-1 accuracy
    top_1_accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test)
    
    # Calculate fully connected accuracy
    fc_accuracy = model.evaluate(X_test, [y_test_cat, y_test_cat], verbose=0)[1]
    
    # Calculate embedding similarity
    similarities = calculate_similarity(embeddings, embeddings)
    
    return top_1_accuracy, fc_accuracy, similarities

if __name__ == "__main__":
    train_dir = r"C:\Users\hp\Desktop\ml\dataset\train"
    test_dir = r"C:\Users\hp\Desktop\ml\dataset\val"
    
    # Load and preprocess data
    X_train, y_train, class_names = load_data(train_dir)
    X_test, y_test, _ = load_data(test_dir)
    
    num_classes = len(class_names)
    
    # Convert labels to categorical
    y_train_cat = to_categorical(y_train, num_classes)
    y_test_cat = to_categorical(y_test, num_classes)
    
    # Create and compile the model
    model = create_model(num_classes)
    
    # Train the model
    model, history = train_model(model, train_dir, test_dir)
    
    # Save the model
    model.save('face_rec_advanced.h5')
    
    # Evaluate the model
    top_1_accuracy, fc_accuracy, similarities = evaluate_model(model, X_test, y_test, num_classes)
    
    print(f"Top-1 Accuracy: {top_1_accuracy:.4f}")
    print(f"Fully Connected Accuracy: {fc_accuracy:.4f}")
    print("Embedding Similarities:")
    print(similarities)

Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 25s 7s/step - classification_accuracy: 0.2985 - classification_top_1_accuracy: 0.2985 - loss: 1.5937 - val_classification_accuracy: 0.3200 - val_classification_top_1_accuracy: 0.3200 - val_loss: 1.5346
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - classification_accuracy: 0.4375 - classification_top_1_accuracy: 0.4375 - loss: 1.5484 - val_classification_accuracy: 0.5600 - val_classification_top_1_accuracy: 0.5600 - val_loss: 1.4792
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - classification_accuracy: 0.6667 - classification_top_1_accuracy: 0.6667 - loss: 1.4285 - val_classification_accuracy: 0.7200 - val_classification_top_1_accuracy: 0.7200 - val_loss: 1.3248
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - classification_accuracy: 0.8621 - classification_top_1_accuracy: 0.8621 - loss: 1.3095 - val_classification_accuracy: 0.6800 - val_classification_top_1_accurac

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Top-1 Accuracy: 0.8000
Fully Connected Accuracy: 0.8000
Embedding Similarities:
[[ 1.0000002   0.77094626  0.51415807  0.3062411   0.6927728   0.4831488
   0.33051085  0.14298996  0.6647019   0.20563939  0.6429514   0.5069053
   0.5900402   0.47949895  0.703919    0.46234637 -0.01424095  0.07460039
   0.16512197  0.23846084  0.2900459   0.08409538 -0.02808679 -0.00631722
  -0.14280717]
 [ 0.77094626  1.0000001   0.5863014   0.26832145  0.726564    0.64916915
   0.45329073  0.304592    0.71574616  0.49395856  0.8462175   0.46289203
   0.71053684  0.6578344   0.87934965  0.35708103 -0.06515568 -0.02243525
   0.17228475  0.1351164   0.10774911 -0.18607126 -0.22437494 -0.15238003
  -0.343748  ]
 [ 0.51415807  0.5863014   1.         -0.02042066  0.5418078   0.7482823
   0.30737972  0.25701007  0.63785934  0.48125488  0.79861677  0.6135055
   0.8160548   0.7894162   0.7859867   0.15526858 -0.11503872 -0.09944816
  -0.02475942  0.13299485  0.03806318 -0.024

In [31]:
# Generate embeddings for a sample image
sample_image = r'C:\Users\hp\Desktop\ml\what.jpeg'
img = cv2.imread(sample_image)
img = cv2.resize(img, (224, 224))
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
    
_, embedding = model.predict(img)
print("Sample Image Embedding:")
print(embedding)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Sample Image Embedding:
[[ 4.07301039e-02  1.10394351e-01  8.91054943e-02  8.16061273e-02
   9.33263730e-03  9.99042168e-02  9.26797241e-02 -3.97027656e-02
  -5.50999977e-02 -1.02053978e-01 -3.90345640e-02  6.07660227e-03
  -2.34638993e-02  1.07193060e-01 -9.93901026e-03  4.17066226e-03
  -5.90168536e-02 -1.76372554e-03 -1.20428698e-02  6.18927851e-02
   1.22242510e-01  1.87546715e-01  8.60504359e-02  2.33807024e-02
  -7.82775804e-02 -1.62601650e-01  1.88875735e-01 -2.17130277e-02
   3.42852548e-02  9.38646868e-02 -3.43954414e-02  1.13239795e-01
   4.42270190e-02  1.27627656e-01  6.58891574e-02  8.01674873e-02
   1.20469151e-04 -2.15219352e-02 -3.16444249e-03  3.22635360e-02
  -1.22991063e-01 -2.36616120e-01  4.24176231e-02 -5.44497073e-02
   1.34413727e-02  1.19682841e-01  1.26599684e-01  1.56316295e-01
   2.23493557e-02  7.47089908e-02 -1.28981978e-01  8.89300182e-02
  -2.89825033e-02 -6.75112233e-02 -1.20885596e-02 -4.29154374e-02
  -7.47759715e-0